In [548]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
import os
import prepare_flightlog as fl

In [549]:
base_dir = os.path.dirname(os.getcwd())
flightlogs_dir = os.path.join(base_dir,"app", "data", "flightlogs")
files = [f for f in os.listdir(flightlogs_dir)
         if f.endswith(".igc") or f.endswith(".IGC")]
print(files)

['12025-05-10-XCT-MKA-01.igc', '2025-05-10-XCT-FBA-02.igc', '2025-05-10-XCT-SKA-02.igc', '2025-05-10-XCT-TRE-01.igc', '2025-05-10-XCT-YCA-01.igc', '2025-05-10-XCT-YTO-09.igc']


In [550]:
file_no = 4
file_path = os.path.join(flightlogs_dir, files[file_no])
df = fl.prepare_data(file_path)


e:\Wolf\Belgeler\Projelerim\Paragliding_Flight_Analyses\app\data\flightlogs\2025-05-10-XCT-YCA-01.igc


In [551]:
df.head()

,filename,datetime,pilot,latitude,longitude,gps_altitude_m,pressure_altitude_m,previous_latitude,previous_longitude,distance_from_takeoff_m,...,bearing,delta_bearing,elapsed_time,delta_time,temp,pressure,humidity,dew_point,wind_speed,wind_deg
0,2025-05-10-XCT-YCA-01.igc,2025-05-10 09:24:58,yilmaz cansari,37.326600,37.177967,1095,1132,37.326600,37.178050,35.741463,...,270,17.0,21.0,1.0,26.94,1013.0,23.0,4.07,0.65,238.0
1,2025-05-10-XCT-YCA-01.igc,2025-05-10 09:24:59,yilmaz cansari,37.326583,37.177883,1095,1132,37.326600,37.177967,42.381908,...,255,15.0,22.0,1.0,26.94,1013.0,23.0,4.07,0.65,238.0
2,2025-05-10-XCT-YCA-01.igc,2025-05-10 09:25:00,yilmaz cansari,37.326550,37.177817,1096,1133,37.326583,37.177883,47.593732,...,237,18.0,23.0,1.0,26.94,1013.0,23.0,4.07,0.65,238.0
3,2025-05-10-XCT-YCA-01.igc,2025-05-10 09:25:01,yilmaz cansari,37.326500,37.177783,1096,1133,37.326550,37.177817,50.223459,...,207,30.0,24.0,1.0,26.94,1013.0,23.0,4.07,0.65,238.0
4,2025-05-10-XCT-YCA-01.igc,2025-05-10 09:25:02,yilmaz cansari,37.326417,37.177767,1097,1133,37.326500,37.177783,52.521371,...,189,18.0,25.0,1.0,26.94,1013.0,23.0,4.07,0.65,238.0


In [552]:
def calc_zscore(df) -> pd.Series:
    mean = df.mean()
    std = df.std()
    result = (df - mean) / std
    result = np.where(abs(result) > 3, True, False)
    return result


df = df[df['gps_altitude_m'] > 0]

zsore_columns = ['gps_altitude_m', 'distance_m', 'speed_km/s',
                 'climb_m', 'climb_m(delta)', 'climb_rate_m/s',
                 'glide_ratio', 'delta_time']

for col in zsore_columns:
    df[col + "_zscore"] = calc_zscore(df[col].copy())
    df = df[df[col + "_zscore"] == False]
    df.drop(columns=[col + "_zscore"], inplace=True)


file_name = files[file_no].split(".")[0]
flight_date = df["datetime"].dt.date[0]
pilot_name = df["pilot"].iloc[0]
max_altitude = df["gps_altitude_m"].max()
max_speed = df["speed_km/s"].max()
total_distance = df["distance_m"].sum()
max_distance = df["distance_from_takeoff_m"].max()
total_time = df["datetime"].iloc[-1] - df["datetime"].iloc[0]
max_elapsed_time = df["elapsed_time"].max()


df = df[['latitude', 'longitude',
         'gps_altitude_m', 'distance_m', 'speed_km/s',
         'climb_m', 'climb_m(delta)', 'climb_rate_m/s',
         'glide_ratio', 'bearing', 'delta_bearing',
         'elapsed_time', 'delta_time', 'temp',
         'pressure', 'humidity', 'dew_point',
         'wind_speed', 'wind_deg']]
df.head()

,latitude,longitude,gps_altitude_m,distance_m,speed_km/s,climb_m,climb_m(delta),climb_rate_m/s,glide_ratio,bearing,delta_bearing,elapsed_time,delta_time,temp,pressure,humidity,dew_point,wind_speed,wind_deg
0,37.326600,37.177967,1095,7.385793,26.588855,0.0,-2.0,0.0,0.000000,270,17.0,21.0,1.0,26.94,1013.0,23.0,4.07,0.65,238.0
1,37.326583,37.177883,1095,7.613898,27.410034,0.0,-2.0,0.0,0.000000,255,15.0,22.0,1.0,26.94,1013.0,23.0,4.07,0.65,238.0
2,37.326550,37.177817,1096,6.971226,25.096413,1.0,-1.0,1.0,6.971226,237,18.0,23.0,1.0,26.94,1013.0,23.0,4.07,0.65,238.0
3,37.326500,37.177783,1096,6.286613,22.631806,0.0,-1.0,0.0,0.000000,207,30.0,24.0,1.0,26.94,1013.0,23.0,4.07,0.65,238.0
4,37.326417,37.177767,1097,9.365868,33.717126,0.0,-1.0,0.0,0.000000,189,18.0,25.0,1.0,26.94,1013.0,23.0,4.07,0.65,238.0


In [553]:
# --- 1. Load saved scaler and Transformer model ---
scaler_path = os.path.join('data', 'models', 'scaler.pkl')
model_path  = os.path.join('data', 'models', 'transformer_tunned_model.keras')

with open(scaler_path, 'rb') as f:
    scaler = pickle.load(f)

model = tf.keras.models.load_model(model_path)
print(f"Loaded Transformer model from '{model_path}'")


# Define feature columns (everything except the target)
feature_cols = [
    'gps_altitude_m', 'distance_m', 'speed_km/s',
    'glide_ratio', 'bearing', 'delta_bearing',
    'temp', 'pressure', 'humidity', 'dew_point',
    'wind_speed', 'wind_deg'
]


Loaded Transformer model from 'data\models\transformer_tunned_model.keras'


In [554]:
# --- 3. Scale features ---
X_new = df[feature_cols].values.astype(np.float32)
X_new_scaled = scaler.transform(X_new)

# --- 4. Create sliding‐window dataset for inference ---
T = 10        # same window length as training
batch_size = 32

predict_ds = tf.keras.preprocessing.timeseries_dataset_from_array(
    data=X_new_scaled,
    targets=None,
    sequence_length=T,
    sequence_stride=1,
    shuffle=False,
    batch_size=batch_size
)

In [555]:
# --- 5. Run predictions ---
preds = model.predict(predict_ds).flatten()

# --- 6. Align predictions with original data indices or timestamps ---
# If your DataFrame has a timestamp column, you can use that instead.
# number of windows = N – T + 1
preds = model.predict(predict_ds).flatten()
# Align preds to df_new by taking index[T-1:]
indices = df.index[T-1:]
# Just in case, truncate to the same length:
indices = indices[: len(preds)]

indices = df.index[T-1 : T-1 + len(preds)]
df.loc[indices, 'predicted_climb_rate_m/s'] = preds

503/503 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [556]:
df = df[df['predicted_climb_rate_m/s'].notna()]
df = df[df['climb_rate_m/s'] > -1]
df["error"] = df['predicted_climb_rate_m/s'] - df['climb_rate_m/s'] 
df.head()

,latitude,longitude,gps_altitude_m,distance_m,speed_km/s,climb_m,climb_m(delta),climb_rate_m/s,glide_ratio,bearing,...,elapsed_time,delta_time,temp,pressure,humidity,dew_point,wind_speed,wind_deg,predicted_climb_rate_m/s,error
10,37.325983,37.177733,1102,9.365869,33.717128,1.0,-1.0,1.0,9.365869,189,...,31.0,1.0,26.94,1013.0,23.0,4.07,0.65,238.0,-0.047507,-1.047507
11,37.325917,37.177733,1103,7.398917,26.636103,0.0,0.0,0.0,0.000000,180,...,32.0,1.0,26.94,1013.0,23.0,4.07,0.65,238.0,0.110757,0.110757
12,37.325850,37.177717,1104,7.544933,27.161759,0.0,0.0,0.0,0.000000,191,...,33.0,1.0,26.94,1013.0,23.0,4.07,0.65,238.0,-0.114307,-0.114307
14,37.325733,37.177717,1103,5.549188,19.977076,0.0,-1.0,0.0,0.000000,180,...,35.0,1.0,26.94,1013.0,23.0,4.07,0.65,238.0,-0.099553,-0.099553
19,37.325333,37.177667,1096,11.484868,41.345526,1.0,-5.0,1.0,11.484868,194,...,40.0,1.0,26.94,1013.0,23.0,4.07,0.65,238.0,-0.221631,-1.221631


In [557]:
y_true = df['climb_rate_m/s'].values
y_pred = df['predicted_climb_rate_m/s'].values
errors = y_true - y_pred

# 2) Pearson korelasyon (−1…1 → map 0…1)
pearson_r = np.corrcoef(y_true, y_pred)[0,1]
pearson_sim = (pearson_r + 1) / 2
pearson_percent = pearson_sim * 100
print(f"Pearson similarity: {pearson_percent:.2f}%")

Pearson similarity: 70.85%


In [558]:
def calc_score(row):
    error = row['climb_rate_m/s'] - row['predicted_climb_rate_m/s']
    score = ((error + 1 ) / 2)*0.80
    timebonus = (row['elapsed_time'] / 14400) * 0.20
    return (score + timebonus)*100

df["score"] = df.apply(calc_score, axis=1)
df.head()

,latitude,longitude,gps_altitude_m,distance_m,speed_km/s,climb_m,climb_m(delta),climb_rate_m/s,glide_ratio,bearing,...,delta_time,temp,pressure,humidity,dew_point,wind_speed,wind_deg,predicted_climb_rate_m/s,error,score
10,37.325983,37.177733,1102,9.365869,33.717128,1.0,-1.0,1.0,9.365869,189,...,1.0,26.94,1013.0,23.0,4.07,0.65,238.0,-0.047507,-1.047507,81.943346
11,37.325917,37.177733,1103,7.398917,26.636103,0.0,0.0,0.0,0.000000,180,...,1.0,26.94,1013.0,23.0,4.07,0.65,238.0,0.110757,0.110757,35.614152
12,37.325850,37.177717,1104,7.544933,27.161759,0.0,0.0,0.0,0.000000,191,...,1.0,26.94,1013.0,23.0,4.07,0.65,238.0,-0.114307,-0.114307,44.618103
14,37.325733,37.177717,1103,5.549188,19.977076,0.0,-1.0,0.0,0.000000,180,...,1.0,26.94,1013.0,23.0,4.07,0.65,238.0,-0.099553,-0.099553,44.030712
19,37.325333,37.177667,1096,11.484868,41.345526,1.0,-5.0,1.0,11.484868,194,...,1.0,26.94,1013.0,23.0,4.07,0.65,238.0,-0.221631,-1.221631,88.920776


In [559]:

df[["climb_rate_m/s", "predicted_climb_rate_m/s", "score", "error"]].head(10)

,climb_rate_m/s,predicted_climb_rate_m/s,score,error
10,1.0,-0.047507,81.943346,-1.047507
11,0.0,0.110757,35.614152,0.110757
12,0.0,-0.114307,44.618103,-0.114307
14,0.0,-0.099553,44.030712,-0.099553
19,1.0,-0.221631,88.920776,-1.221631
21,0.0,-0.076645,43.124140,-0.076645
22,1.0,0.065343,77.446008,-0.934657
23,0.0,0.019374,39.286165,0.019374
24,1.0,-0.382823,95.375427,-1.382823
25,0.0,-1.064573,82.646811,-1.064573


In [ ]:
print(f"Uçuş adı: {file_name}")
print(f"Uçuş tarihi: {flight_date}")
print(f"Pilot adı: {pilot_name}")
print(f"En yüksek irtifa: {max_altitude:.0f} m")
print(f"En yüksek hız: {max_speed:.2f} km/s")
print(f"Toplam mesafe: {total_distance/1000:.2f} km")
print(f"En uzun mesafe: {max_distance/1000:.2f} km")
print(f"Pearson similarity: {pearson_percent:.2f}%")
print(f"Performans skoru: {df['score'].mean():.2f}%")
print(f"Toplam süre: {total_time}")

Uçuş adı: 2025-05-10-XCT-YCA-01
Uçuş tarihi: 2025-05-10
Pilot adı: yilmaz cansari
En yüksek irtifa: 3776 m
En yüksek hız: 83.78 km/s
Toplam mesafe: 209.14 km
En uzun mesafe: 142.16 km
Pearson similarity: 70.85%
Bu uçuş için performans skoru: 71.45%
Toplam süre: 0 days 04:33:40
